<a href="https://colab.research.google.com/github/yeonjin99/TIL/blob/main/Search_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. LangChain을 사용하여 문장 임베딩, 벡터 스토어, 문서 검색을 설정하기
- 텍스트 데이터에서 임베딩을 생성하고, 임베딩을 바탕으로 특정 질의에 맞는 문서를 검색하는 것

In [ ]:
!pip install langchain openai docarray tiktoken

In [ ]:
!pip install cohere

In [ ]:
# import modules required
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

#### 임베딩 및 벡터 스토어 생성
- 아래 문장들에 대해 **임베딩(Embedding)** 생성 -> 벡터 스토어에 저장
- 벡터 스토어는 문장을 벡터 형식으로 저장한 다음, 사용자가 입력한 질의에 따라 유사도를 기반으로 가장 유사한 문장을 검색

In [ ]:
# 임베딩 및 벡터 스토어 생성
vectorstore = DocArrayInMemorySearch.from_texts(
    ["There are 4 seasons in Korea",
     "Harry Potter is an outstanding wizard",
     "The dog likes to walk"],
    embedding=OpenAIEmbeddings(),
)

In [ ]:
# 검색기 세팅
retriever = vectorstore.as_retriever()

In [ ]:
# 검색기 실행
retriever.invoke("Who is Harry Potter?")

## 2. LangChain을 사용한 벡터 기반 검색 & OpenAI 모델을 통한 질의 응답 생성

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### 1. 단순한 영어 질문에 대한 답변

In [ ]:
# 벡터 스토어 생성 및 검색기 생성
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 생성
#   -> 사용자가 질문에 대해 답변할 때 필요한 컨텍스트와 질문을 받아, OpenAI 모델에게 전달할 포맷을 구성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# 모델 및 체인 설정
model = ChatOpenAI()

# 체인은 여러 단계로 구성된 파이프라인
#   -> context: retriever에서 검색된 문장을 전달
#   -> question: 사용자가 입력한 질문을 전달(RunnablePassthrough는 변환 없이 전달)
#   -> StrOutputParser: 모델의 출력을 문자열로 파싱
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# 체인 실행
chain.invoke("where did harrison work?")

### 2. 여러 언어 중 하나로 답변

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# 다국어 체인 실행
chain.invoke({"question": "where did harrison work", "language": "italian"})